In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer

# Define repressilator model
def model(variables, t, params):

    m1, p1, m2, p2, m3, p3 = variables
    k1, k2, k3 = params #only ks are parameters to infer
    a1 = a2 = a3 = 24.78485282457379
    g1 = g2 = g3 = 0.024884149937163258
    n1 = n2 = n3 = 5
    b1 = b2 = b3 = 33.82307682700831
    dm1 = dm2 = dm3 = 1.143402097500176
    dp1 = dp2 = dp3 = 0.7833664565550977

    dm1dt = -dm1 * m1 + (a1 / (1 + ((1/k1) * p2) ** n1)) + g1
    dp1dt = (b1 * m1) - (dp1 * p1)
    dm2dt = -dm2 * m2 + (a2 / (1 + ((1/k2) * p3) ** n2)) + g2
    dp2dt = (b2 * m2) - (dp2 * p2)
    dm3dt = -dm3 * m3 + (a3 / (1 + ((1/k3) * p1) ** n3)) + g3
    dp3dt = (b3 * m3)-(dp3 * p3)
    
    return [dm1dt, dp1dt, dm2dt, dp2dt, dm3dt, dp3dt]

# Define the true parameters for k
true_params = np.array([
    246.96291990024542, 246.96291990024542, 246.96291990024542,])

# Establish prior from 0-250
num_dim = 3
prior = utils.BoxUniform(low=10**-2 * torch.ones(num_dim), high=250 * torch.ones(num_dim))

In [ ]:
# We define the time variable (0-100 in 1000 steps), the initial conditions, etc.
num_timesteps = 1000
num_trajectories = 6
initial_conditions = np.array([0, 1, 0, 3, 0, 2])
t = np.linspace(0, 100, num_timesteps)


t = np.linspace(0, 100, num_timesteps)
def my_simulator(theta):
    initial_conditions = np.array([0, 2, 0, 1, 0, 3], dtype=np.float32)
    solution = odeint(model, initial_conditions, t, args=(theta,))
    return torch.tensor(solution, dtype=torch.float32)  # Flatten tensor to size [600]

x_o = my_simulator(true_params)
x_o.shape

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size #Number of GRU units
        self.input_size = input_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)   
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Number of layers, batch size = 2, number of GRU units
        gru_out, _ = self.gru(x, h0)                # GRU layer, we don't store the hn (hidden state output)
        output = self.linear(gru_out[:, -1, :])  #(N,L,D∗H_out when batch_first=True containing the output features (h_t) from the last layer of the RNN, for each t.
        return output

# Define RNN
input_size = 6
sequence_length=1000
output_size = 25 #We want 1-dimensional embeddings of length = 25 (1x25)
hidden_size = 100 #GRU units

#reshaped data should be [batch, sequence_length, input_size] - [batch, 1000,6]
embedding_net = RNN(input_size, hidden_size, output_size)

Testing out shape

In [ ]:
import torch

# Define the input tensor shape [batch_size, sequence_length, input_size]
dummy_input = torch.randn(2, sequence_length, input_size)  # Assuming batch size of 2
# Pass the input through the embedding network
output = embedding_net(dummy_input)
# Print the output size
print("Output size:", output.size())

In [ ]:
# The prepare for sbi function checks simulator and prior are compatible with the NPE algorithm (and makes sure it adds a batch dimension)
simulator_wrapper, prior = prepare_for_sbi(my_simulator, prior)
theta, x = simulate_for_sbi(simulator_wrapper, prior, num_simulations=10) # Simulate data
print(theta.shape,x.shape)


In [ ]:

# We define the neural network (neural density estimator), specifying the embedding net. In this case we use a mixture density network.
neural_posterior = utils.posterior_nn(
    model="mdn", embedding_net=embedding_net, hidden_features=10, num_transforms=2)

# Setup the inference procedure with the SNPE-C (Greenberg et al, 2019)
inference = SNPE(prior=prior, density_estimator=neural_posterior)

density_estimator = inference.append_simulations(theta, x).train(training_batch_size=2)        # Train density estimator
posterior = inference.build_posterior(density_estimator)       

In [ ]:
# Then sample the posterior and plot
posterior_samples = posterior.sample((100,), x=x_o)
_ = analysis.pairplot(
    posterior_samples, limits=[[-100, 300], [-100, 300], [-100, 300]], figsize=(6, 6) 
)